In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Installing Nvidia Apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a

/opt/conda/lib/python3.6/site-packages/pip/_internal/commands/install.py:244: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-b2vkwxvx
Created temporary directory: /tmp/pip-req-tracker-vm4i3lev
Created requirements tracker '/tmp/pip-req-tracker-vm4i3lev'
Created temporary directory: /tmp/pip-install-wn7ld7mx
Processing /kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a
  Created temporary directory: /tmp/pip-req-build-1_4vypcf
  Added file:///kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a to build tracker '/tmp/pip-req-tracker-vm4i3lev'
    Running setup.py (path:/tmp/pip-req-build-1_4vypcf/setup.py) egg_info for package from file:///kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a
    Running command python setup.py egg_info
    torch.__version__  =  1.1.0
    running egg_info
    creating pip-egg

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil

In [4]:
device=torch.device('cuda')

In [5]:
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
EPOCHS = 1
Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification"
Input_dir = "../input"
WORK_DIR = "../working/"
TOXICITY_COLUMN = 'target'
num_to_load=1000000

In [6]:
!pip install allennlp
!pip install git+https://github.com/Strideradu/pytorch-pretrained-BERT.git --upgrade

     |████████████████████████████████| 1.7MB 13.2MB/s 
     |████████████████████████████████| 51kB 25.9MB/s 
     |████████████████████████████████| 5.6MB 42.5MB/s 
ERROR: sacred 0.7.5 has requirement colorama>=0.4, but you'll have colorama 0.3.9 which is incompatible.
  Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0
  Found existing installation: colorama 0.4.1
    Uninstalling colorama-0.4.1:
      Successfully uninstalled colorama-0.4.1
  Found existing installation: botocore 1.12.173
    Uninstalling botocore-1.12.173:
      Successfully uninstalled botocore-1.12.173
  Cloning https://github.com/Strideradu/pytorch-pretrained-BERT.git to /tmp/pip-req-build-64mi7l9z
  Running command git clone -q https://github.com/Strideradu/pytorch-pretrained-BERT.git /tmp/pip-req-build-64mi7l9z
  Stored in directory: /tmp/pip-ephem-wheel-cache-2xhkndbp/wheels/46/e3/f7/ea842a7d6f66c57e9d2d0b08638455df95e087f718c35be562
Successfully built pyto

In [7]:
# Add the Bart Pytorch repo to the PATH
# using files from: https://github.com/huggingface/pytorch-pretrained-BERT
from pytorch_pretrained_bert import XLNetTokenizer, XLNetForClassification

In [8]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-large-cased")

100%|██████████| 798011/798011 [00:00<00:00, 1887771.89B/s]


In [9]:
model = XLNetForClassification.from_pretrained("xlnet-large-cased", clf_dropout=0.4, n_class=8)

100%|██████████| 467/467 [00:00<00:00, 69887.61B/s]


In [10]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    #print(example)
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        #print(tokens_a)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["<cls>"]+tokens_a+["<eod>"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [11]:
%%time
train_df = pd.read_csv(os.path.join(Data_dir,"train.csv")).sample(10000,random_state=SEED)
print('loaded %d records' % len(train_df))
train_df=train_df.fillna(0)

loaded 10000 records
CPU times: user 10.9 s, sys: 2.01 s, total: 12.9 s
Wall time: 13 s


In [12]:
sequences = convert_lines(train_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)


335


In [13]:
# convert target to 0,1
y_aux_train = train_df[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]

In [14]:
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

# Overall
weights = np.ones((len(train_df),)) / 4

# Subgroup
weights += (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4

# Background Positive, Subgroup Negative
weights += (( (train_df['target'].values>=0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4

# Background Negative, Subgroup Positive
weights += (( (train_df['target'].values<0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
loss_weight = 1.0 / weights.mean()

y_train = np.vstack([(train_df['target'].values>=0.5).astype(np.int),weights]).T

In [15]:
targets = np.hstack([y_train, y_aux_train])

In [16]:
targets.shape

(10000, 8)

In [17]:
X = sequences              
y = targets

In [18]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))

In [19]:
def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,1:])
    return (bce_loss_1 * loss_weight) + bce_loss_2

In [20]:
def children(m):
    return m if isinstance(m, (list, tuple)) else list(m.children())

def set_trainable_attr(m, b):
    m.trainable = b
    for p in m.parameters():
        p.requires_grad = b


def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, torch.nn.Module):
        f(m)
    if len(c) > 0:
        for l in c:
            apply_leaf(l, f)

            
def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m, b))

In [21]:
del X
del y
del sequences
del targets
del train_df

gc.collect()

856

In [22]:
from torch.optim import Adam

In [23]:
output_model_file = "XlNet_pytorch.bin"

lr=2e-5
batch_size = 6
accumulation_steps = 1
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


model.zero_grad()
model = model.to(device)
train = train_dataset

#num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)
optimizer = Adam(params=model.parameters(), 
                 lr=lr,
                 eps=1e-6,   
                 amsgrad=False)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model=model.train()
tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad()   # Bug fix - thanks to @chinhuic
    for i,(x_batch, y_batch) in tk0:
        y_pred = model(x_batch.to(device))
        loss = custom_loss(y_pred, y_batch.to(device))
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)

torch.save(model.state_dict(), output_model_file)